In [42]:
import numpy as np
import pandas as pd
import scipy.stats as stats
from scipy.stats import ttest_ind
import seaborn as sns

https://mipt-stats.gitlab.io/courses/python/07_scipy_stats.html

In [3]:
 """Оцениваем ошибку первого рода.

    Бутстрепим выборки из пилотной и контрольной групп тех же размеров, считаем долю случаев с значимыми отличиями.
    
    df_pilot_group - pd.DataFrame, датафрейм с данными пилотной группы
    df_control_group - pd.DataFrame, датафрейм с данными контрольной группы
    metric_name - str, названия столбца с метрикой
    alpha - float, уровень значимости для статтеста
    n_iter - int, кол-во итераций бутстрапа
    seed - int or None, состояние генератора случайных чисел.

    return - float, ошибка первого рода"""

'Оцениваем ошибку первого рода.\n\n   Бутстрепим выборки из пилотной и контрольной групп тех же размеров, считаем долю случаев с значимыми отличиями.\n   \n   df_pilot_group - pd.DataFrame, датафрейм с данными пилотной группы\n   df_control_group - pd.DataFrame, датафрейм с данными контрольной группы\n   metric_name - str, названия столбца с метрикой\n   alpha - float, уровень значимости для статтеста\n   n_iter - int, кол-во итераций бутстрапа\n   seed - int or None, состояние генератора случайных чисел.\n\n   return - float, ошибка первого рода'

In [70]:
a = np.random.normal(0, 1, 10)
b = np.random.normal(0.3, 1, 10)

In [71]:
a = pd.DataFrame(a)
b = pd.DataFrame(b)

сгенерить датафреймы 
найти интервал - три вида статистики 
посчитать случаи, когда > alpha
оценить вероятность ошибки первого и второго рода 

In [73]:
def estimate_first_type_error(df_pilot_group, df_control_group, name = str, alpha=0.05, n_iter=1000, seed=None):
    
    def check_ttest(a, b, alpha=alpha):
        """Тест Стьюдента. Возвращает 1, если отличия значимы."""
        _, pvalue = stats.ttest_ind(a, b)
        return int(pvalue < alpha)
    

    def check_mannwhitneyu(a, b, alpha=alpha):
        """Тест Манн-Уитни. Возвращает 1, если отличия значимы."""
        _, pvalue = stats.mannwhitneyu(a, b, alternative='two-sided')
        return int(pvalue < alpha)


    def check_bootstrap(a, b, func=np.mean, B=n_iter, alpha=alpha):
        """Бутстрап. Возвращает 1, если отличия значимы."""
        #переведем датафрем в series
        a= a.iloc[:, 0] 
        b = b.iloc[:, 0] 
        # если попадает внутрь довертельного интервала, то возвращает 0 и различия не значимы и наоборот 
        a_bootstrap = np.random.choice(a, size=(len(a), B))
        b_bootstrap = np.random.choice(b, size=(len(b), B))
        list_diff = func(a_bootstrap, axis=0) - func(b_bootstrap, axis=0)
        left_bound = np.quantile(list_diff, alpha / 2)
        right_bound = np.quantile(list_diff, 1 - alpha / 2)
        res = 1 if (left_bound > 0) or (right_bound < 0) else 0
        return res


    dict_tests = {
        'ttest': check_ttest,
        'mannwhitneyu': check_mannwhitneyu,
        'bootstrap': check_bootstrap}
    
    first_type_errors = []
    second_type_errors = []
    sample_size = 0.5
    ab_results = [] #складываем результаты аb тестов 
    
    for _ in range(n_iter):
        a =  df_control_group.sample(frac=sample_size)
        b = df_pilot_group.sample(frac=sample_size)
        test_yes_no = dict_tests[name](a, b)
        ab_results.append(test_yes_no)

    first_type_errors.append(sum(ab_results)/len(ab_results))
    second_type_errors.append(1 - sum(ab_results)/len(ab_results))

    return first_type_errors, second_type_errors

In [75]:
estimate_first_type_error(a, b, name = 'ttest', alpha=0.05, n_iter=1000, seed=None)

([0.029], [0.971])